In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re
import numpy as np

In [2]:
#Open firefox and go to url
driver = webdriver.Firefox()

In [3]:
def find_number_class(driver):
    #Get region where the number and class are stored
    class_region = driver.find_element(By.CLASS_NAME, 'col-md-10')
    
    #get the number
    punk_title = class_region.find_element(By.XPATH, './/h1')
    #use regex to extract only the numbers
    numbers_pattern = r'\d+'
    punk_number = re.search(numbers_pattern, punk_title.text)
    
    #get the class
    punk_class = class_region.find_element(By.XPATH, './/h4//a')
    
    return punk_number.group(), punk_class.text

In [4]:
def find_atts(section): #make a long string containing all attributes
    atts = section.find_elements(By.CLASS_NAME, 'col-md-4')
    att_string = ''
    
    for count, element in enumerate(atts):
        
        attribute = element.find_element(By.XPATH, './/a')
        
        if count == 0:
            att_string = att_string + attribute.text
        
        else : 
            att_string = att_string + ',' + attribute.text
    
    return att_string

In [5]:
def find_current_owner(section):
    current_owner_region = section.find_element(By.CLASS_NAME, 'col-md-10')
    
    return current_owner_region.find_element(By.XPATH, './/div//a').text

In [6]:
def punkToFile(punk, fileName):
    
    file = open(fileName, 'a', encoding = 'utf-8')
        
    file.write(punk['number'] + ';' + punk['class'] + ';' + punk['attributes'] + ';'\
                   + punk['current_owner'] + ';' + '\n')
    #If we do not close the file, the file will not be saved
    file.close()

In [7]:
def find_write_transactions(section, punk_number):
    #get the table
    trans_table = section.find_element(By.CLASS_NAME, 'table')
    #get the rows
    trans_rows = trans_table.find_elements(By.XPATH, './/tbody//tr')
    
    file = open(str(punk_number) + '.txt', 'x', encoding = 'utf-8')
    for row in trans_rows:
        info = row.find_elements(By.XPATH, './/td')
        info_string = ''
        
        #get each bit of info
        for count, bit in enumerate(info):
            #if it is the first field, no need to add the separator
            if count == 0:
                info_string = info_string + bit.text
                
            #if it is the fourth field, it is the price
            elif count == 3:
                # if the string is void, the match does not work
                if bit.text != '':
                    #print(bit.text)
                    #get the price
                    pattern = r'\$\<*\>*\d+,*.*\d*K*M*B*'
                    new_str = re.search(pattern, bit.text).group()
                    
                    #get rid of the last parenthesis and the possible thousand comma
                    number_str = new_str[1:-1].replace(',' , '')
                    
                    #if the string end with a K, multiply by 1000
                    if number_str[-1] == 'K':
                        #change the portion of the string to a float, then multiply
                        #then change back to string
                        number_str = str(float(number_str[:-1]) * 1000)
                    
                    #same, with millions
                    elif number_str[-1] == 'M':
                        number_str = str(float(number_str[:-1]) * 1000000)
                        
                    #same with billions
                    elif number_str[-1] == 'B':
                        number_str = str(float(number_str[:-1]) * 1000000000)
                    
                    #add the separator plus the string
                    info_string = info_string + ';' + number_str
                
                #for empty fields, just add the separator. We need this else,
                # otherwise the separator would not be added
                else:
                    info_string = info_string + ';'
                    
            #the rest of the fields have no special cases
            else :
                info_string = info_string + ';' + bit.text
                
        #write the string, then newline        
        file.write(info_string + '\n')
    file.close()

In [ ]:
for number in range(80,2500):
    url = 'https://www.larvalabs.com/cryptopunks/details/'
    driver.get(url + str(number))
    driver.implicitly_wait(np.random.random()*10)
    punk = {}
    punk['number'], punk['class'] = find_number_class(driver)
    
    #get different sections where info is actually stored
    sections = driver.find_elements(By.CLASS_NAME, 'row')
    
    punk['attributes'] = find_atts(sections[4])
    
    punk['current_owner'] = find_current_owner(sections[6])
    
    punkToFile(punk, 'Punks.txt')
    
    find_write_transactions(sections[7], punk['number'])
    #TODO change date format. This should be changed 
    #in the function    

0.10Ξ ($375)
0.10Ξ ($323)
<0.01Ξ ($3)
<0.01Ξ ($<0.01)
0.25Ξ ($778)
0.25Ξ ($778)
115Ξ ($414,735)
115Ξ ($413,261)
10Ξ ($18,086)
10Ξ ($18,091)
8Ξ ($14,555)
8Ξ ($14,606)
6Ξ ($11,022)
6Ξ ($11,022)
0.50Ξ ($1,568)
0.50Ξ ($1,568)
0.05Ξ ($159)
0.05Ξ ($156)
0.03Ξ ($75)
0.02Ξ ($37)
25Ξ ($37,472)
25Ξ ($38,957)
1.25Ξ ($306)
1.25Ξ ($305)
33Ξ ($82,040)
30Ξ ($58,972)
22.88Ξ ($44,976)
22.88Ξ ($47,764)
8.80Ξ ($11,289)
8.80Ξ ($11,240)
3Ξ ($2,237)
3Ξ ($1,884)
0.15Ξ ($142)
0.15Ξ ($142)
0.05Ξ ($38)
0.10Ξ ($31)
0.10Ξ ($38)
890Ξ ($1.59M)
300Ξ ($542,466)
51Ξ ($70,660)
51Ξ ($76,069)
47Ξ ($67,925)
47Ξ ($67,178)
45Ξ ($67,954)
175Ξ ($265,400)
45Ξ ($68,246)
41Ξ ($63,172)
150Ξ ($218,316)
41Ξ ($60,274)
35Ξ ($55,515)
35Ξ ($57,603)
250Ξ ($433,628)
16Ξ ($5,443)
19.99Ξ ($6,898)
16Ξ ($5,521)
19.99Ξ ($6,898)
20Ξ ($4,138)
45Ξ ($9,352)
20Ξ ($4,137)
50Ξ ($10,366)
2Ξ ($2,301)
0.99Ξ ($301)
1Ξ ($308)
1.35Ξ ($386)
1.80Ξ ($510)
2.49Ξ ($732)
2.50Ξ ($869)
3Ξ ($895)
3.50Ξ ($1,038)
3.99Ξ ($1,229)
3.39Ξ ($766)
3.49Ξ ($703)
3.50Ξ ($705)

In [10]:
mystr = '0.10Ξ ($375)'
pattern = r'\$\d+,*.*\d*K*M*B*'
new_str = re.search(pattern, mystr).group()
print(new_str)

$375)
